In [1]:
# Import pandas
import pandas as pd
from pandas import Series, DataFrame
pd.set_option('display.max_columns', 500)

# Import other util
import time
import datetime
import re

In [2]:
special_character = '，。？：“『』｜＋＿（）＊＆＾％＄＃＠！～｀‘'
replace_rule = '[0-9a-zA-Z_]|\W|' + '|'.join(special_character)
replace_rule

'[0-9a-zA-Z_]|\\W|，|。|？|：|“|『|』|｜|＋|＿|（|）|＊|＆|＾|％|＄|＃|＠|！|～|｀|‘'

### 教育部辭典
- 當中含有奇怪的字
    - gif 檔名
    - 字間不乾淨 (e.g. `聞人\u30002.名士`, `回復\u3000`)
- 移除長度為 1 的詞

In [3]:
edu_dict = set()
alias_dict = {}

In [4]:
def extract_edu_dict(df, word_col, alias_col):
    global edu_dict
    global alias_dict
    edu_dict.update([
        re.sub(replace_rule, '', word)
        for word in df[word_col].values
        if len(word) > 1 and 'gif' not in word and 'jpg' not in word and 'png' not in word
        ])
    for _, row in df.loc[df[alias_col].notnull(), [word_col, alias_col]].iterrows():
        word = row[word_col]
        alias = row[alias_col]
        if not (len(word) > 1 and 'gif' not in word and 'jpg' not in word and 'png' not in word):
            continue
        word = re.sub(replace_rule, '', word)
        alias = re.sub(replace_rule, ' ', alias)
        if word not in alias_dict:
            alias_dict[word] = set()
        alias_dict[word].update(alias.split())

In [5]:
df1 = pd.read_excel('datas/raw/dict_concised_2014_20150204.xls')
df1

,字詞號,字詞名,部首,部首外筆畫數,總筆畫數,注音一式,漢語拼音,相似詞,相反詞,釋義,編按,多音參見訊息
0,0001,八,八,0,2,ㄅㄚ,bā,NaN,NaN,1.介於七與九之間的自然數。大寫作「捌」，阿拉伯數字作「8」。 \n2.表示數量是八的...,（「八」字口語連用在去聲字前讀成陽平。&9b51._104_0.gif;八號、八拜）,NaN
1,000100000,八拜之交,NaN,0,0,ㄅㄚ ㄅㄞˋ ㄓ ㄐㄧㄠ,bā bài zhī jiāo,NaN,NaN,稱結拜為異姓兄弟姊妹的朋友。古時因為對父執輩需行八拜禮，所以朋友若相交親密如同手足，經過約定...,NaN,NaN
2,0001000001,八寶飯,NaN,0,0,ㄅㄚ ㄅㄠˇ ㄈㄢˋ,bā bǎo fàn,NaN,NaN,一種甜點。以適量的糯米、蓮子、紅棗、黑棗、紅豆沙、桂圓肉、葡萄乾、糖、豬油（或植物油）等材料...,NaN,NaN
3,000100001,八寶粥,NaN,0,0,ㄅㄚ ㄅㄠˇ ㄓㄡ,bā bǎo zhōu,NaN,NaN,用糯米、蓮子、桂圓肉、紅棗、冬瓜糖、葡萄乾、豬油、糖等多種材料煮成的粥品。&9b51._10...,NaN,NaN
4,000100002,八面玲瓏,NaN,0,0,ㄅㄚ ㄇㄧㄢˋ ㄌㄧㄥˊ ㄌㄨㄥˊ,bā miàn líng lóng,NaN,NaN,形容人處世圓滑，面面俱到。&9b51._104_0.gif;他為人處世八面玲瓏，很吃得開。\n,NaN,NaN
5,0001000021,八面威風,NaN,0,0,ㄅㄚ ㄇㄧㄢˋ ㄨㄟ ㄈㄥ,bā miàn wēi fēng,NaN,NaN,比喻聲勢浩大、神氣十足的樣子。&9b51._104_0.gif;我國代表隊的王牌投手在冠亞軍...,NaN,NaN
6,000100003,八方,NaN,0,0,ㄅㄚ ㄈㄤ,bā fāng,NaN,NaN,東、南、西、北、東南、西南、東北、西北八個方向的總稱，一般泛指所有的方向。&9b51._10...,NaN,NaN
7,000100004,八德,NaN,0,0,ㄅㄚ ㄉㄜˊ,bā dé,NaN,NaN,忠、孝、仁、愛、信、義、和、平八種美德。&9b51._104_0.gif;四維八德\n,NaN,NaN
8,000100005,八年抗戰,NaN,0,0,ㄅㄚ ㄋㄧㄢˊ ㄎㄤˋ ㄓㄢˋ,bā nián kàng zhàn,NaN,NaN,民國二十六年七七事變後至三十四年間，抵抗日本對華侵略的戰爭。經過八年的艱苦奮戰，我國終於獲得...,NaN,NaN
9,000100006,八哥,NaN,0,0,ㄅㄚ ㄍㄜ,bā gē,NaN,NaN,動物名。鳥綱燕雀目八哥科。毛色純黑，頭及背部微帶綠色光澤。頭上羽毛細長而尖，呈柳葉狀。善於模...,NaN,NaN


In [6]:
df1.loc[df1['相似詞'].notnull(), '相似詞'].values

array(['&8e4c._104_0.gif;頂尖', '&8e4c._104_0.gif;戲法',
       '&8e4c._104_0.gif;1.而已', ..., '&8e4c._104_0.gif;輸送、運送',
       '&8e4c._104_0.gif;輸送、運輸', '&8e4c._104_0.gif;應用'], dtype=object)

In [7]:
extract_edu_dict(df1, '字詞名', '相似詞')
len(edu_dict), len(alias_dict)

(38270, 3764)

In [8]:
df2 = pd.read_excel('datas/raw/dict_revised_2015_20160523_1.xls')
df2

,字詞屬性,字詞號,字詞名,部首字,部首外筆畫數,總筆畫數,注音一式,漢語拼音,相似詞,相反詞,釋義,編按,多音參見訊息,異體字
0,1,00001,八,八,0,2,ㄅㄚ,bā,NaN,NaN,[名]\n1.介於七與九之間的自然數。如：「六、七、八、九……」。大寫作「捌」，阿拉伯數字作...,（「八」字口語連用在去聲字前讀成陽平，如：「八號」、「八拜」。）,NaN,0
1,1,00002,扒,手,2,5,(一)ㄅㄚ,(一)bā,NaN,NaN,[動] \n1.剝開。如：「把橘子扒開來吃。」 \n2.扯掉。如：「扒衣裳」。 ...,NaN,(二)ㄆㄚˊ pá（00584）,0
2,1,00003,叭,口,2,5,ㄅㄚ,bā,NaN,NaN,[名]\n參見「喇叭」條。\n[狀]\n形容汽車、機車的喇叭聲。如：「忽聽叭的一聲，嚇了我一...,NaN,NaN,0
3,1,00004,巴,己,1,4,ㄅㄚ,bā,NaN,NaN,[名]\n1.國名。春秋時所建的國家，故城約在今大陸地區四川省東部。\n2.量詞。計算大氣壓...,NaN,NaN,0
4,1,00005,吧,口,4,7,(一)ㄅㄚ,(一)bā,NaN,NaN,[名]\n酒吧。為英文 bar的音譯。如：「吧檯」、「吧女」。\n[動]\n吸（菸）。如：「...,NaN,(二)˙ㄅㄚ ba（00036）,0
5,1,00006,芭,艸,4,8,ㄅㄚ,bā,NaN,NaN,[名]\n1.一種香草。《楚辭．屈原．九歌．禮魂》：「成禮兮會鼓，傳芭兮代舞。」宋．洪興祖．...,NaN,NaN,0
6,1,00007,疤,疒,4,9,ㄅㄚ,bā,NaN,NaN,[名]\n1.瘡口或傷口癒合後所留下的痕跡。明．張自烈《正字通．疒部》：「疤，俗呼瘡痕曰：『...,NaN,NaN,0
7,1,00008,羓,羊,4,10,ㄅㄚ,bā,NaN,NaN,[名]\n1.乾肉。《集韻．平聲．麻韻》：「羓，腊屬。」如：「羓子」、「羊羓」。《新五代史．...,NaN,NaN,0
8,1,00009,粑,米,4,10,ㄅㄚ,bā,NaN,NaN,[名]\n譯音用字。參見「糌粑」條。\n[綴]\n表乾扁的食物。如：「鍋粑」。,NaN,NaN,0
9,1,00010,笆,竹,4,10,ㄅㄚ,bā,NaN,NaN,[名]\n1.一種有刺的竹子，即棘竹。《玉篇．竹部》：「笆，竹有刺。」\n2.用竹枝或竹片編...,NaN,NaN,0


In [9]:
df2.loc[df2['相似詞'].notnull(), '相似詞'].values

array(['四行孤軍', '2.八面見光、面面俱到、面面俱圓、左右逢源', '八面玲瓏、左右逢源', ..., '豐功偉績', '功勞、成績',
       '移玉、枉駕'], dtype=object)

In [10]:
extract_edu_dict(df2, '字詞名', '相似詞')
len(edu_dict), len(alias_dict)

(74038, 7306)

In [11]:
df3 = pd.read_excel('datas/raw/dict_revised_2015_20160523_2.xls')
df3

,字詞屬性,字詞號,字詞名,部首字,部首外筆畫數,總筆畫數,注音一式,漢語拼音,相似詞,相反詞,釋義,編按,多音參見訊息,異體字
0,2,032610061,勞金,NaN,0,0,ㄌㄠˊ ㄐㄧㄣ,láo jīn,NaN,NaN,1.酬金。如：「老李一向熱心公益，從不收取任何勞金。」\n2.工錢。如：「這是你這個月的勞金...,NaN,NaN,0
1,2,032610062,勞倦,NaN,0,0,ㄌㄠˊ ㄐㄩㄢˋ,láo juàn,NaN,NaN,勞累、疲倦。《史記．卷一一四．東越傳》：「上曰士卒勞倦，不許，罷兵。」《紅樓夢．第二回》：「...,NaN,NaN,0
2,2,032610063,勞怯,NaN,0,0,ㄌㄠˊ ㄑㄩㄝˋ,láo què,NaN,NaN,肺病。舊時認為肺病是縱情酒色，身體疲勞過度所引起，故稱為「勞怯」。《二刻拍案驚奇．卷二一》：...,NaN,NaN,0
3,2,032610064,勞心,NaN,0,0,ㄌㄠˊ ㄒㄧㄣ,láo xīn,NaN,1.2.勞力,1.用腦力工作。《孟子．滕文公上》：「勞心者治人，勞力者治於人。」\n2.費神、傷神。如：「...,NaN,NaN,0
4,2,032610065,勞心焦思,NaN,0,0,ㄌㄠˊ ㄒㄧㄣ ㄐㄧㄠ ㄙ,láo xīn jiāo sī,NaN,NaN,形容人為事情擔憂煩心。唐．杜甫〈憶昔〉詩二首之一：「至今今上猶撥亂，勞心焦思補四方。」也作「...,NaN,NaN,0
5,2,032610066,勞形,NaN,0,0,ㄌㄠˊ ㄒㄧㄥˊ,láo xíng,NaN,NaN,身體因事情煩亂而感到疲勞。《莊子．應帝王》：「是於聖人也，胥易技係，勞形怵心者也。」唐．劉禹...,NaN,NaN,0
6,2,032610067,勞成,NaN,0,0,ㄌㄠˊ ㄔㄥˊ,láo chéng,NaN,NaN,殷勤、體貼，多指虛情假意。元．朱庭玉〈夜行船．無限鶯花慵管領套．天仙令〉曲：「千金廢，火上弄...,NaN,NaN,0
7,2,032610068,勞承,NaN,0,0,ㄌㄠˊ ㄔㄥˊ,láo chéng,NaN,NaN,殷勤、體貼，多指虛情假意。元．喬吉《兩世姻緣．第二折》：「緊緊的將咱摟定，那溫存，那將惜，那...,NaN,NaN,0
8,2,032610069,勞師動眾,NaN,0,0,ㄌㄠˊ ㄕ ㄉㄨㄥˋ ㄓㄨㄥˋ,láo shī dòng zhòng,NaN,NaN,動員眾多的人力。《紅樓夢．第四五回》：「只怕我們家裡還有，與你送幾兩來，每日叫丫頭們就熬了，...,NaN,NaN,0
9,2,032610070,勞師襲遠,NaN,0,0,ㄌㄠˊ ㄕ ㄒㄧˊ ㄩㄢˇ,láo shī xí yuǎn,NaN,NaN,發動軍隊襲擊遠方的敵人。語出《左傳．僖公三十二年》：「勞師以襲遠，非所聞也。」多用以指冒險、...,NaN,NaN,0


In [12]:
df3.loc[df3['相似詞'].notnull(), '相似詞'].values

array(['費心、費神', '徒勞無益、心勞日拙、吃力不討好、枉用心機', '絮聒', ..., '3.核心、焦點', '核心分子、重要分子',
       '停止、停滯、中斷'], dtype=object)

In [13]:
extract_edu_dict(df3, '字詞名', '相似詞')
len(edu_dict), len(alias_dict)

(118124, 11519)

In [14]:
df4 = pd.read_excel('./datas/raw/dict_revised_2015_20160523_3.xls')
df4

,字詞屬性,字詞號,字詞名,部首字,部首外筆畫數,總筆畫數,注音一式,漢語拼音,相似詞,相反詞,釋義,編按,多音參見訊息,異體字
0,2,081300425,中輟,NaN,0,0,ㄓㄨㄥ ㄔㄨㄛˋ,zhōng chuò,NaN,NaN,中途停頓。《文選．潘岳．笙賦》：「舞既蹈而中輟，節將撫而弗及。」《五代史平話．周史．卷下》：...,NaN,NaN,0
1,2,081300427,中川,NaN,0,0,ㄓㄨㄥ ㄔㄨㄢ,zhōng chuān,NaN,NaN,江中、河中。南朝宋．謝靈運〈登江中孤嶼〉詩：「亂流趨孤嶼，孤嶼媚中川。」唐．柳宗元〈遊南亭夜...,NaN,NaN,0
2,2,081300428,中船,NaN,0,0,ㄓㄨㄥ ㄔㄨㄢˊ,zhōng chuán,NaN,NaN,中國造船股份有限公司的縮稱。為國營生產事業單位。其中高雄總廠為國家十大建設之一，於民國六十五...,NaN,NaN,0
3,2,081300429,中使,NaN,0,0,ㄓㄨㄥ ㄕˇ,zhōng shǐ,NaN,NaN,內廷的使者，多指宦官。《南齊書．卷二六．王敬則傳》：「中使相望，軒冕成陰。」《初刻拍案驚奇．...,NaN,NaN,0
4,2,081300430,中士,NaN,0,0,ㄓㄨㄥ ㄕˋ,zhōng shì,NaN,NaN,1.中等的人。《老子．第四一章》：「中士聞道，若存若亡。」\n2.職官名。祿位在上士之下，下...,NaN,NaN,0
5,2,081300431,中世,NaN,0,0,ㄓㄨㄥ ㄕˋ,zhōng shì,NaN,NaN,1.中古時代。《韓非子．五蠹》：「上古競於道德，中世逐於智謀，當今爭於氣力。」《後漢書．卷四...,NaN,NaN,0
6,2,081300432,中世紀,NaN,0,0,ㄓㄨㄥ ㄕˋ ㄐㄧˋ,zhōng shì jì,NaN,NaN,歐洲歷史上指自西元四七六年西羅馬帝國滅亡，至一六四○英國資產階級革命這一段時期，此時期歐洲處...,NaN,NaN,0
7,2,081300435,中式,NaN,0,0,(一)ㄓㄨㄥ ㄕˋ,zhōng shì,NaN,NaN,中國傳統的式樣。如：「中式餐點」。,NaN,(二)ㄓㄨㄥˋ ㄕˋ zhòng shì（081500037）,0
8,2,081300440,中視,NaN,0,0,ㄓㄨㄥ ㄕˋ,zhōng shì,NaN,NaN,中國電視公司的縮稱。參見「中國電視公司」條。,NaN,NaN,0
9,2,081300441,中沙群島,NaN,0,0,ㄓㄨㄥ ㄕㄚ ㄑㄩㄣˊ ㄉㄠˇ,zhōng shā qún dǎo,NaN,NaN,群島名。我國南海諸島中四大群島之一，隸屬於海南島。位於西沙群島東南，分布略成橢圓形。為一群尚...,NaN,NaN,0


In [15]:
df4.loc[df4['相似詞'].notnull(), '相似詞'].values

array(['正午', '漢文、華文', '不偏不倚', ..., '回佣', '回佣', '2.多數'], dtype=object)

In [16]:
extract_edu_dict(df4, '字詞名', '相似詞')
len(edu_dict), len(alias_dict)

(151703, 14740)

In [17]:
# Preview edu_dict
edu_dict

{'詞句',
 '泥垢',
 '五顏六色',
 '任由',
 '亂彈',
 '麥克羅尼西亞人',
 '千般萬樣',
 '少年觀護所',
 '護衛艦',
 '兩頭春',
 '起花頭',
 '留後步',
 '呆頭木腦',
 '宿酒',
 '過過身',
 '苦思惡想',
 '淚竹',
 '隧道棚栽法',
 '細節',
 '利路',
 '詳悉',
 '首座',
 '琉璃磚',
 '題本',
 '活用',
 '口過',
 '合指症',
 '羊質虎皮',
 '窮措大',
 '踏踏實實',
 '草衣木食',
 '驅口',
 '朝梁暮陳',
 '悶錶',
 '床頭金盡',
 '矜育',
 '二傳',
 '客帳司',
 '負號',
 '不得地',
 '討愧',
 '柏翳',
 '疙瘩事',
 '語料庫',
 '效法',
 '好個',
 '聖旨',
 '疲塌',
 '兩廂',
 '一語破的',
 '監市履狶',
 '巧作',
 '趙翼',
 '無計可施',
 '仙霞嶺',
 '輪值',
 '拼貼',
 '排中律',
 '談麈',
 '量小易盈',
 '餒怯',
 '小額',
 '梨乾兒',
 '欣抃',
 '不叫',
 '賜姓',
 '浴鐵',
 '心內膜炎',
 '重重的',
 '作癟子',
 '個性難改',
 '車馬輻輳',
 '一心無罣礙',
 '原點',
 '討命',
 '困阻',
 '一天星斗',
 '傳述',
 '發瘋',
 '斗食',
 '河海',
 '規範',
 '青田石',
 '事假',
 '差價',
 '屁股',
 '歸計',
 '就坐',
 '柏酒',
 '糜子',
 '廚下',
 '年近花甲',
 '觸鬚',
 '暢行',
 '冰碴兒',
 '大義滅親',
 '矢言',
 '一緊二慢三罷休',
 '書富五車',
 '有力',
 '疼惜',
 '徂謝',
 '西塔琴',
 '寓意',
 '切膚之痛',
 '生花',
 '浮奢',
 '金穴',
 '鴿派',
 '有色人種',
 '響板',
 '剗地',
 '小姑獨處',
 '茂發',
 '西北颱',
 '搖滾樂',
 '德言工貌',
 '衰謝',
 '捫參歷井',
 '套項',
 '雲芝',
 '陽明大學',
 '藥典',
 '黃雀伺蟬',
 '互通有無',
 '蛋家',
 '登山驀嶺',
 '劫寨',
 '原是'

In [18]:
# Preview alias_dict
alias_dict

{'憫惻': {'哀憐'},
 '詞句': {'文句'},
 '患難夫妻': {'同命鴛鴦'},
 '五顏六色': {'五光十色', '五彩繽紛'},
 '到手': {'得手'},
 '爬山': {'登山'},
 '相識': {'認識'},
 '名義': {'名堂'},
 '任由': {'任憑'},
 '解危': {'抒難', '紓難'},
 '落拓不羈': {'不拘小節'},
 '清冷': {'清涼'},
 '匾打側卓': {'旁敲側擊'},
 '父老': {'長者', '長輩'},
 '宰衡': {'宰相', '宰輔', '首相'},
 '麻木': {'麻痺', '麻醉'},
 '醉心': {'傾慕'},
 '交友': {'相交'},
 '身手': {'技藝', '武藝'},
 '反目': {'不和', '反面'},
 '良策': {'上策', '善策'},
 '浩劫': {'大難'},
 '大權獨攬': {'大權在握'},
 '墮淚': {'哭泣', '隕涕'},
 '昨日': {'昨天'},
 '好逸惡勞': {'吊兒郎當', '游手好閑', '遊手好閒', '養尊處優'},
 '好菜': {'佳餚'},
 '捉住': {'抓住'},
 '籍籍無名': {'沒沒無聞'},
 '瞎攪': {'亂來'},
 '嚚猾': {'奸詐'},
 '飾物': {'首飾'},
 '挾勢': {'仗勢'},
 '博覽群書': {'博古知今', '博古通今', '博覽古今'},
 '瞽者': {'盲人', '瞎子'},
 '突然': {'乍然', '倏忽', '忽然', '猛然', '猝然', '驀地', '驟然'},
 '笨蛋': {'傻瓜', '笨伯'},
 '真誠': {'真摯', '竭誠', '誠實', '誠摯'},
 '牛之一毛': {'九牛一毫', '滄海一粟'},
 '時光': {'光陰', '時間', '韶光'},
 '簡直': {'幾乎'},
 '發回': {'發還'},
 '耽溺': {'沉溺', '沉迷'},
 '公允': {'公平', '公正'},
 '踏踏實實': {'腳踏實地'},
 '查辦': {'查究', '核辦'},
 '改革': {'改良', '改變', '改造', '改革', '變革', '釐革', '革新'},
 '末路': {'末日',

In [19]:
# Using disjoint set create alias mapper
disjoint_set_rooter = {}

def find_root(word):
    global disjoint_set_rooter
    if word not in disjoint_set_rooter:
        disjoint_set_rooter[word] = word
    if disjoint_set_rooter[word] == word:
        return word
    disjoint_set_rooter[word] = find_root(disjoint_set_rooter[word])
    return disjoint_set_rooter[word]

def union_set(word1, word2):
    global disjoint_set_rooter
    word1 = find_root(word1)
    word2 = find_root(word2)
    disjoint_set_rooter[word2] = word1

In [20]:
for key, vals in alias_dict.items():
    for v in vals:
        union_set(key, v)
for key in list(alias_dict.keys()):
    disjoint_set_rooter[key] = find_root(key)

In [21]:
disjoint_set_rooter

{'憫惻': '憫惻',
 '詞句': '詞句',
 '患難夫妻': '患難夫妻',
 '五顏六色': '五花八門',
 '到手': '到手',
 '爬山': '爬山',
 '相識': '攪渾',
 '名義': '攪渾',
 '任由': '非論',
 '解危': '解危',
 '落拓不羈': '囚首垢面',
 '氣味相投': '意氣相合',
 '清冷': '風涼',
 '口紅': '脣膏',
 '匾打側卓': '匾打側卓',
 '宰衡': '輔弼',
 '麻木': '麻木',
 '莊稼': '農事',
 '交友': '結交',
 '身手': '攪渾',
 '反目': '不睦',
 '良策': '良策',
 '浩劫': '浩劫',
 '蘼蕪路斷': '秋扇見捐',
 '大權獨攬': '大權獨攬',
 '墮淚': '流淚',
 '昨日': '昨日',
 '好逸惡勞': '游手好閒',
 '好菜': '美饌',
 '捉住': '捉住',
 '籍籍無名': '籍籍無名',
 '違法亂紀': '明火執仗',
 '瞎攪': '瞎攪',
 '嚚猾': '權詐',
 '飾物': '細軟',
 '平放': '平鋪',
 '好天': '晴天',
 '挾勢': '挾勢',
 '博覽群書': '博學多聞',
 '瞽者': '瞽者',
 '突然': '溘然',
 '敕令': '攪渾',
 '笨蛋': '笨蛋',
 '真誠': '攪渾',
 '牛之一毛': '牛之一毛',
 '專斷獨行': '固執己見',
 '時光': '攪渾',
 '簡直': '險些',
 '發回': '發回',
 '耽溺': '攪渾',
 '濟困解危': '濟困扶危',
 '公允': '合理',
 '踏踏實實': '踏踏實實',
 '查辦': '攪渾',
 '改革': '變造',
 '算作': '當作',
 '末路': '死胡同',
 '專家': '各人',
 '得到': '得到',
 '萬種': '各式',
 '白麵': '白粉',
 '長壽': '長壽',
 '效法': '仿傚',
 '臘鼓頻催': '臘鼓頻催',
 '麻木不仁': '如臨深淵',
 '聖旨': '聖旨',
 '對抗': '攪渾',
 '告白': '告白',
 '風燭殘年': '風燭殘年',
 '蠻幹': '蠻幹',
 '無計可施': '如臨深淵',


In [27]:
# Output result
with open('datas/dict/edu_dict.txt', 'w') as f:
    f.write('\n'.join(list(edu_dict)))

In [33]:
with open('datas/dict/edu_alias.txt', 'w') as f:
    _ = dict([(v, []) for v in set(disjoint_set_rooter.values())])
    for k, v in disjoint_set_rooter.items():
        _[v].append(k)
    for k, v in _.items()

憫惻
